<a href="https://colab.research.google.com/github/Aliciak363/21century_web_scrape/blob/main/scraping_century21_web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup

In [4]:
r=requests.get("https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/")
c=r.content
soup=BeautifulSoup(c, 'html.parser')
#print(soup.prettify())

In [23]:
all=soup.find_all('div',{'class':'propertyRow'})
all[0].find('h4', {'class': 'propPrice'}).text.replace('\n', "").replace(" ", "")

page_nr=soup.find_all('a', {'class': 'Page'})[-1].text   #[-1] = last type

In [24]:
l=[]

base_url="https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s="   #url for all three pages
for page in range(0, int(page_nr)*10, 10):  #0 az 30, a delene 10 kazda strana
    print(base_url + str(page) + ".html")
    r=requests.get(base_url + str(page) + ".html")
    c=r.content
    soup=BeautifulSoup(c, "html.parser")
    all=soup.find_all('div',{'class':'propertyRow'})

    for item in all:
      d={}         
      d['Address']=item.find_all('span', {'class': 'propAddressCollapse'})[0].text                      
      d['Locality']=item.find_all('span', {'class': 'propAddressCollapse'})[1].text     
      d['Price']=item.find('h4', {'class': 'propPrice'}).text.replace('\n', "").replace(" ", "")      

      try:
          d['Beds']=item.find('span', {'class': 'infoBed'}).text   # + (.find('b').text)  
      except: 
          d['Beds']=None                    

      try:
          d['Baths']=item.find('span', {'class': 'infoValueFullBath'}).text
      except: 
          d['Baths']=None

      try:
          d['Area']=item.find('span', {'class': 'infoSqFt'}).text
      except: 
          d['Area']=None  

      for column_group in item.find_all('div', {'class': 'columnGroup'}):
          for feature_group, feature_name in zip(column_group.find_all('span', {'class': 'featureGroup'}), column_group.find_all('span', {'class': 'featureName'})):
              if 'Lot Size' in feature_group.text:      #'Lot Size' = Acre(s)
                  d['Lot Size']=feature_name.text
      l.append(d)

https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=0.html
https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=10.html
https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=20.html


In [25]:
import pandas
df=pandas.DataFrame(l)
df

,Address,Locality,Price,Beds,Baths,Area,Lot Size
0,0 Gateway,"Rock Springs, WY 82901","$725,000",None,None,None,NaN
1,1003 Winchester Blvd.,"Rock Springs, WY 82901","$452,900",4 Beds,4 Full Baths,None,0.21 Acres
2,600 Talladega,"Rock Springs, WY 82901","$396,900",5 Beds,3 Full Baths,"3,154 Sq. Ft",NaN
3,3239 Spearhead Way,"Rock Springs, WY 82901","$389,900",4 Beds,3 Full Baths,"3,076 Sq. Ft","Under 1/2 Acre,"
4,522 Emerald Street,"Rock Springs, WY 82901","$254,000",3 Beds,3 Full Baths,"1,172 Sq. Ft","Under 1/2 Acre,"
5,1302 Veteran's Drive,"Rock Springs, WY 82901","$252,900",4 Beds,2 Full Baths,"1,932 Sq. Ft",0.27 Acres
6,1021 Cypress Cir,"Rock Springs, WY 82901","$210,000",4 Beds,3 Full Baths,"1,676 Sq. Ft","Under 1/2 Acre,"
7,913 Madison Dr,"Rock Springs, WY 82901","$209,000",3 Beds,2 Full Baths,"1,344 Sq. Ft","Under 1/2 Acre,"
8,1344 Teton Street,"Rock Springs, WY 82901","$199,900",3 Beds,2 Full Baths,"1,920 Sq. Ft","Under 1/2 Acre,"
9,4 Minnies Lane,"Rock Springs, WY 82901","$196,900",3 Beds,2 Full Baths,"1,664 Sq. Ft",2.02 Acres


In [26]:
df.to_csv('info_houses.csv')